# OpenSMILE Analysis
This notebook loads OpenSMILE csv- data, cleans and plots it

## Import relevant libraries

In [1]:
import numpy as np
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt
import itertools as it
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.api as sm
#import nltk
import scipy.stats as st

import seaborn as sns
import Helper as hp

## Load .csv data with results of OpenSMILE Analysis
First we load .csv data and clean it (removing of NaNs), then we store information of all files in seperate panda dataframes containing information about affect, emotion and valence/arousal for all participants.

In [2]:
# Get Filenames to iterate
filenames = listdir('OpenSMILE_Data')

#Create python lists which store results
arousal = []
valence = []
emotion = []
affect = []
loi = []
characterIDs = []

fnames = []
filenames.sort()
for x in filenames:
    fnames.append([x])    
    
emotion_label = ['anger', 'boredom', 'disgust', 'fear', 'happiness', 'neutral', 'sadness']
affect_label = ['aggressiv', 'cheerful', 'intoxicated', 'nervous', 'neutral', 'tired']
loi_label = ['disinterest', 'normal', 'high interest']

for i in range(len(fnames)):
     # data contains all information (arousal, valence, emotion, affect) and we want to save the values of all files in a list
    data = pd.read_csv("OpenSMILE_Data/" + filenames[i])    

    # For arousal, valence and affect we have to drop nans, since they have less values than emotion
    temp_arousal = data['arousal']
    temp_arousal = temp_arousal.dropna()
    temp_valence = data['valence'] 
    temp_valence = temp_valence.dropna()
    temp_affect = data['abcAffect']
    temp_affect = temp_affect.dropna()
    temp_emotion = data['emodbEmotion']
    temp_loi = data['avicLoI']
    temp_loi = temp_loi.dropna()
    characterIDs.append(fnames[i][0][0])
    
    #Append the temp values to 'global lists'
    emotion.append(temp_emotion.values.tolist())
    affect.append(temp_affect.values.tolist())
    valence.append(temp_valence.values)
    arousal.append(temp_arousal.values)
    loi.append(temp_loi.values)

#We want to have the labels as column seperators and the filenames as ID 
#We want to do this, so that if we add more files (at the moment only 6 .csv files are loaded) we want to add rows and not columns
#If we plot the data, emotion_label can be used as label
df_emotion = pd.DataFrame.from_records(emotion)
df_emotion.columns = emotion_label
df_emotion['CharacterID'] = characterIDs
df_emotion['file'] = filenames

#Now do the same for affect
df_affect = pd.DataFrame.from_records(affect)
df_affect.columns = affect_label
df_affect['CharacterID'] = characterIDs
df_affect['file'] = filenames

#Now for loi
df_loi = pd.DataFrame.from_records(loi)
df_loi.columns = loi_label
df_loi['CharacterID'] = characterIDs
df_loi['file'] = filenames

#For Arousal and Valence, we want to combine these two features so that we can draw a scatter plot in the arousal valence space
np_ar = np.array(arousal).ravel()
np_val = np.array(valence).ravel()
np_ar_val = np.array([np_ar, np_val])

#Transpose Matrix so that it is in the same form as affect and emotion (columns = arousal, valence, ID = Filename)
df_ar_val = pd.DataFrame.from_records(np_ar_val.T)
#df_ar_val.index = filenames
df_ar_val.columns = ['valence', 'arousal']
df_ar_val['CharacterID'] = characterIDs
df_ar_val['file'] = filenames

## Let's load information about the speakers
The speaker ID is saved in a single .csv file containing four important columns: ID, Age, Sex and Acadedmic Status. Since before loaded OpenSMILE csv files are named using the corresponding index (ex. speaker with id 0 has two files 0_a.csv and 0_b.csv), so that a link can be created

In [3]:
char_data = pd.read_csv("CharacterData/CharacterIDs.csv")    

#Join above tables and Character Tables

#To Join DataFrames we have to cast the column on which we want to join to int, so that both columns have the same data type
char_data['CharacterID'] = char_data['CharacterID'].astype(int)
df_ar_val['CharacterID'] = df_ar_val['CharacterID'].astype(int)
df_emotion['CharacterID'] = df_emotion['CharacterID'].astype(int)
df_affect['CharacterID'] = df_affect['CharacterID'].astype(int)
df_loi['CharacterID'] = df_loi['CharacterID'].astype(int)

#Safe new data frames
df_ar_val_char = df_ar_val.merge(char_data, how = 'left', left_on='CharacterID', right_on='CharacterID')
df_emotion_char = df_emotion.merge(char_data, how = 'left', left_on='CharacterID', right_on= 'CharacterID')
df_affect_char = df_affect.merge(char_data, how = 'left', left_on='CharacterID', right_on= 'CharacterID')
df_loi_char = df_loi.merge(char_data, how = 'left', left_on='CharacterID', right_on= 'CharacterID')

## Now have a look at correlation
### Have a look at arousal and what it corrlates with

In [4]:
#'Clean' our dataFrames so that we can call the panda .corr function (default is spearman)
#Let's start with arousal and look how correlation is between emotions
print('Emotions and Arousal')
corr_ar_e_anger = hp.correlation(df_ar_val['arousal'],df_emotion['anger'])
coh_d_ar_e_anger = hp.cohen_d(df_ar_val['arousal'], df_emotion['anger'])

corr_ar_e_boredom = hp.correlation(df_ar_val['arousal'], df_emotion['boredom'])
coh_d_ar_e_bordedom = hp.cohen_d(df_ar_val['arousal'], df_emotion['boredom'])

corr_ar_e_disgust = hp.correlation(df_ar_val['arousal'], df_emotion['disgust'])
coh_d_ar_e_disgust = hp.cohen_d(df_ar_val['arousal'], df_emotion['disgust'])
corr_ar_e_fear = hp.correlation(df_ar_val['arousal'], df_emotion['fear'])
coh_ar_e_fear = hp.cohen_d(df_ar_val['arousal'], df_emotion['fear'])
corr_ar_e_happy = hp.correlation(df_ar_val['arousal'], df_emotion['happiness'])
coh_ar_e_happy = hp.cohen_d(df_ar_val['arousal'], df_emotion['happiness'])
corr_ar_e_neutral = hp.correlation(df_ar_val['arousal'], df_emotion['neutral'])
coh_ar_e_neutral = hp.cohen_d(df_ar_val['arousal'], df_emotion['neutral'])
corr_ar_e_sad = hp.correlation(df_ar_val['arousal'], df_emotion['sadness'])
coh_ar_e_sad = hp.cohen_d(df_ar_val['arousal'], df_emotion['sadness'])

# Now arousal and affect
print('Affect and Arousal')
corr_ar_a_aggressiv = hp.correlation(df_ar_val['arousal'],df_affect['aggressiv'])
coh_d_ar_a_aggressiv = hp.cohen_d(df_ar_val['arousal'],df_affect['aggressiv'])
corr_ar_a_cheerful = hp.correlation(df_ar_val['arousal'], df_affect['cheerful'])
coh_d_ar_a_cheerful = hp.cohen_d(df_ar_val['arousal'],df_affect['cheerful'])
corr_ar_a_intoxicated = hp.correlation(df_ar_val['arousal'], df_affect['intoxicated'])
coh_d_ar_a_intoxicated = hp.cohen_d(df_ar_val['arousal'],df_affect['intoxicated'])
corr_ar_a_nervous = hp.correlation(df_ar_val['arousal'], df_affect['nervous'])
coh_d_ar_a_nervous = hp.cohen_d(df_ar_val['arousal'],df_affect['nervous'])
corr_ar_a_neutral = hp.correlation(df_ar_val['arousal'], df_affect['neutral'])
coh_d_ar_a_neutral = hp.cohen_d(df_ar_val['arousal'],df_affect['neutral'])
corr_ar_a_tired = hp.correlation(df_ar_val['arousal'],df_affect['tired'])
coh_d_ar_a_tired = hp.cohen_d(df_ar_val['arousal'],df_affect['tired'])


# Now arousal and level of Interest
print('Level of Interest and Arousal')
corr_ar_loi1 = hp.correlation(df_ar_val['arousal'], df_loi['disinterest'])
coh_d_ar_loi1 = hp.cohen_d(df_ar_val['arousal'], df_loi['disinterest'])
corr_ar_loi2 = hp.correlation(df_ar_val['arousal'], df_loi['normal'])
coh_d_ar_loi2 = hp.cohen_d(df_ar_val['arousal'], df_loi['normal'])
corr_ar_loi3 = hp.correlation(df_ar_val['arousal'], df_loi['high interest'])
coh_d_ar_loi3 = hp.cohen_d(df_ar_val['arousal'], df_loi['high interest'])

Emotions and Arousal
Correlation between arousal and anger: -0.6834858044692085
Cohen d: 1.3540103907062906
Correlation between arousal and boredom: -0.42307029758274
Cohen d: 0.1583736511748122
Correlation between arousal and disgust: -0.15806251433006668
Cohen d: -0.7373572423651821
Correlation between arousal and fear: -0.6184722018376901
Cohen d: 1.3418849887875337
Correlation between arousal and happiness: -0.5789869405166657
Cohen d: 1.2675246862010017
Correlation between arousal and neutral: -0.356264634376464
Cohen d: 1.073710483184311
Correlation between arousal and sadness: 0.483425559781051
Cohen d: -2.861793647848958
Affect and Arousal
Correlation between arousal and aggressiv: -0.6249619461578844
Cohen d: -0.05694323448409338
Correlation between arousal and cheerful: -0.7547280735964071
Cohen d: -1.7333878080657694
Correlation between arousal and intoxicated: 0.8192158568074172
Cohen d: -1.3141756512058569
Correlation between arousal and nervous: -0.4817454038193749
Cohen 

### Now look at valence

In [5]:
print('Emotions and valence')
corr_val_e_anger = hp.correlation(df_ar_val['valence'], df_emotion['anger'])
coh_d_val_e_anger = hp.cohen_d(df_ar_val['valence'], df_emotion['anger'])
corr_val_e_boredom = hp.correlation(df_ar_val['valence'], df_emotion['boredom'])
coh_d_val_e_boredom = hp.cohen_d(df_ar_val['valence'], df_emotion['boredom'])
corr_val_e_disgust = hp.correlation(df_ar_val['valence'], df_emotion['disgust'])
coh_d_val_e_disgust = hp.cohen_d(df_ar_val['valence'], df_emotion['disgust'])
corr_val_e_fear = hp.correlation(df_ar_val['valence'], df_emotion['fear'])
coh_d_val_e_fear = hp.cohen_d(df_ar_val['valence'], df_emotion['fear'])
corr_val_e_happy = hp.correlation(df_ar_val['valence'], df_emotion['happiness'])
coh_d_val_e_happy = hp.cohen_d(df_ar_val['valence'], df_emotion['happiness'])
corr_val_e_neutral = hp.correlation(df_ar_val['valence'], df_emotion['neutral'])
coh_d_val_e_neutral = hp.cohen_d(df_ar_val['valence'], df_emotion['neutral'])
corr_val_e_sad = hp.correlation(df_ar_val['valence'], df_emotion['sadness'])
coh_d_val_e_sad = hp.cohen_d(df_ar_val['valence'], df_emotion['sadness'])

# Now valence and affect
print('Affect and valence')
corr_val_a_aggressiv = hp.correlation(df_ar_val['valence'], df_affect['aggressiv'])
coh_val_a_aggressiv = hp.cohen_d(df_ar_val['valence'], df_affect['aggressiv'])
corr_val_a_cheerful = hp.correlation(df_ar_val['valence'], df_affect['cheerful'])
coh_val_a_cheerful = hp.cohen_d(df_ar_val['valence'], df_affect['cheerful'])
corr_val_a_intoxicated = hp.correlation(df_ar_val['valence'],df_affect['intoxicated'])
coh_val_a_intoxicated= hp.cohen_d(df_ar_val['valence'], df_affect['intoxicated'])
corr_val_a_nervous = hp.correlation(df_ar_val['valence'],df_affect['nervous'])
coh_val_a_nervous = hp.cohen_d(df_ar_val['valence'], df_affect['nervous'])
corr_val_a_neutral = hp.correlation(df_ar_val['valence'],df_affect['neutral'])
coh_val_a_neutral = hp.cohen_d(df_ar_val['valence'], df_affect['neutral'])
corr_val_a_tired = hp.correlation(df_ar_val['valence'], df_affect['tired'])
coh_val_a_tired = hp.cohen_d(df_ar_val['valence'], df_affect['tired'])

# Now arousal and level of Interest
print('Level of Interest and valence')
corr_val_loi1 = hp.correlation(df_ar_val['valence'], df_loi['disinterest'])
coh_d_val_loi1 = hp.cohen_d(df_ar_val['valence'], df_loi['disinterest'])
corr_val_loi2 = hp.correlation(df_ar_val['valence'], df_loi['normal'])
coh_d_val_loi2 = hp.cohen_d(df_ar_val['valence'], df_loi['normal'])
corr_val_loi3 = hp.correlation(df_ar_val['valence'], df_loi['high interest'])
coh_d_val_loi3 = hp.cohen_d(df_ar_val['valence'], df_loi['high interest'])

Emotions and valence
Correlation between valence and anger: -0.42700028268595
Cohen d: 2.124212618215266
Correlation between valence and boredom: 0.16831427339329016
Cohen d: 1.198707389958022
Correlation between valence and disgust: -0.17965790687690897
Cohen d: 0.03344577381746021
Correlation between valence and fear: -0.3312140972517545
Cohen d: 2.1168342579669335
Correlation between valence and happiness: -0.3991203323202824
Cohen d: 2.071387756958357
Correlation between valence and neutral: 0.2970377418952006
Cohen d: 1.9499189451425651
Correlation between valence and sadness: 0.12493730716157896
Cohen d: -1.7990379478117744
Affect and valence
Correlation between valence and aggressiv: -0.5966541351575038
Cohen d: 1.1899909839878873
Correlation between valence and cheerful: -0.16359981081171018
Cohen d: -0.18032792829621663
Correlation between valence and intoxicated: 0.3355725268618499
Cohen d: -0.38644640823664206
Correlation between valence and nervous: 0.27861337746066017
Cohe

### Now look at the different emotions and how they correlate with affect and level of interest

In [6]:
# Now emotion and affect
print('ANGER')
print('Affect and anger')
corr_ang_a_aggressiv = hp.correlation(df_emotion['anger'],df_affect['aggressiv'])
coh_d_ang_a_aggressiv = hp.cohen_d(df_emotion['anger'],df_affect['aggressiv'])
corr_ang_a_cheerful = hp.correlation(df_emotion['anger'], df_affect['cheerful'])
coh_d_ang_a_cheerful = hp.cohen_d(df_emotion['anger'],df_affect['cheerful'])
corr_ang_a_intoxicated = hp.correlation(df_emotion['anger'], df_affect['intoxicated'])
coh_d_ang_a_intoxicated = hp.cohen_d(df_emotion['anger'],df_affect['intoxicated'])
corr_ang_a_nervous = hp.correlation(df_emotion['anger'], df_affect['nervous'])
coh_d_ang_a_nervous = hp.cohen_d(df_emotion['anger'],df_affect['nervous'])
corr_ang_a_neutral = hp.correlation(df_emotion['anger'], df_affect['neutral'])
coh_d_ang_a_neutral = hp.cohen_d(df_emotion['anger'],df_affect['neutral'])
corr_ang_a_tired = hp.correlation(df_emotion['anger'], df_affect['tired'])
coh_d_ang_a_tired = hp.cohen_d(df_emotion['anger'],df_affect['tired'])

# Now arousal and level of Interest
print('Level of Interest and anger')
corr_ang_loi1 = hp.correlation(df_emotion['anger'], df_loi['disinterest'])
coh_d_ang_loi1 = hp.cohen_d(df_emotion['anger'], df_loi['disinterest'])
corr_ang_loi2 = hp.correlation(df_emotion['anger'], df_loi['normal'])
coh_d_ang_loi2 = hp.cohen_d(df_emotion['anger'], df_loi['normal'])
corr_ang_loi3 = hp.correlation(df_emotion['anger'], df_loi['high interest'])
coh_d_ang_loi3 = hp.cohen_d(df_emotion['anger'], df_loi['high interest'])

##########
# Now look at boredom
print('BOREDOM')
print('Affect and boredom')
corr_bor_a_aggressiv = hp.correlation(df_emotion['boredom'],df_affect['aggressiv'])
coh_d_bor_a_aggressiv = hp.cohen_d(df_emotion['boredom'],df_affect['aggressiv'])
corr_bor_a_cheerful = hp.correlation(df_emotion['boredom'],df_affect['cheerful'])
coh_d_bor_a_cheerful = hp.cohen_d(df_emotion['boredom'],df_affect['cheerful'])
corr_bor_a_intoxicated = hp.correlation(df_emotion['boredom'],df_affect['intoxicated'])
coh_d_bor_a_intoxicated = hp.cohen_d(df_emotion['boredom'],df_affect['intoxicated'])
corr_bor_a_nervous = hp.correlation(df_emotion['boredom'],df_affect['nervous'])
coh_d_bor_a_nervous = hp.cohen_d(df_emotion['boredom'],df_affect['nervous'])
corr_bor_a_neutral = hp.correlation(df_emotion['boredom'],df_affect['neutral'])
coh_d_bor_a_neutral = hp.cohen_d(df_emotion['boredom'],df_affect['neutral'])
corr_bor_a_tired = hp.correlation(df_emotion['boredom'],df_affect['tired'])
coh_d_bor_a_tired = hp.cohen_d(df_emotion['boredom'],df_affect['tired'])

# Now arousal and level of Interest
print('Level of Interest and boredom')
corr_bor_loi1 = hp.correlation(df_emotion['boredom'], df_loi['disinterest'])
coh_d_bor_loi1 = hp.cohen_d(df_emotion['boredom'], df_loi['disinterest'])
corr_bor_loi2 = hp.correlation(df_emotion['boredom'], df_loi['normal'])
coh_d_bor_loi2 = hp.cohen_d(df_emotion['boredom'], df_loi['normal'])
corr_bor_loi3 = hp.correlation(df_emotion['boredom'], df_loi['high interest'])
coh_d_bor_loi3 = hp.cohen_d(df_emotion['boredom'], df_loi['high interest'])

##########
# Disgust
print('DISGUST')
print('Affect and disgust')
corr_dis_a_aggressiv = hp.correlation(df_emotion['disgust'],df_affect['aggressiv'])
coh_d_dis_a_aggressiv = hp.cohen_d(df_emotion['disgust'],df_affect['aggressiv'])
corr_dis_a_cheerful = hp.correlation(df_emotion['disgust'],df_affect['cheerful'])
coh_d_dis_a_cheerful = hp.cohen_d(df_emotion['disgust'],df_affect['cheerful'])
corr_dis_a_intoxicated = hp.correlation(df_emotion['disgust'],df_affect['intoxicated'])
coh_d_dis_a_intoxicated = hp.cohen_d(df_emotion['disgust'],df_affect['intoxicated'])
corr_dis_a_nervous = hp.correlation(df_emotion['disgust'], df_affect['nervous'])
coh_d_dis_a_nervous = hp.cohen_d(df_emotion['disgust'],df_affect['nervous'])
corr_dis_a_neutral = hp.correlation(df_emotion['disgust'], df_affect['neutral'])
coh_d_dis_a_neutral = hp.cohen_d(df_emotion['disgust'],df_affect['neutral'])
corr_dis_a_tired = hp.correlation(df_emotion['disgust'], df_affect['tired'])
coh_d_dis_a_tired = hp.cohen_d(df_emotion['disgust'],df_affect['tired'])

# Now level of Interest
print('Level of Interest and disgust')
corr_dis_loi1 =  hp.correlation(df_emotion['disgust'],df_loi['disinterest'])
coh_d_dis_loi1 = hp.cohen_d(df_emotion['disgust'],df_loi['disinterest'])
corr_dis_loi2 =  hp.correlation(df_emotion['disgust'],df_loi['normal'])
coh_d_dis_loi2 = hp.cohen_d(df_emotion['disgust'],df_loi['normal'])
corr_dis_loi3 =  hp.correlation(df_emotion['disgust'],df_loi['high interest'])
coh_d_dis_loi3 = hp.cohen_d(df_emotion['disgust'],df_loi['high interest'])

######
# fear
print('FEAR')
print('Affect and fear')
corr_fea_a_aggressiv = hp.correlation(df_emotion['fear'], df_affect['aggressiv'])
coh_d_fea_a_aggressiv = hp.cohen_d(df_emotion['fear'], df_affect['aggressiv'])
corr_fea_a_cheerful = hp.correlation(df_emotion['fear'],df_affect['cheerful'])
coh_d_fea_a_cheerful = hp.cohen_d(df_emotion['fear'], df_affect['cheerful'])
corr_fea_a_intoxicated = hp.correlation(df_emotion['fear'],df_affect['intoxicated'])
coh_d_fea_a_intoxicated = hp.cohen_d(df_emotion['fear'], df_affect['intoxicated'])
corr_fea_a_nervous = hp.correlation(df_emotion['fear'],df_affect['nervous'])
coh_d_fea_a_nervous = hp.cohen_d(df_emotion['fear'], df_affect['nervous'])
corr_fea_a_neutral = hp.correlation(df_emotion['fear'], df_affect['neutral'])
coh_d_fea_a_neutral = hp.cohen_d(df_emotion['fear'], df_affect['neutral'])
corr_fea_a_tired = hp.correlation(df_emotion['fear'],df_affect['tired'])
coh_d_fea_a_tired = hp.cohen_d(df_emotion['fear'], df_affect['tired'])

# Now arousal and level of Interest
print('Level of Interest and fear')
corr_fea_loi1 = hp.correlation(df_emotion['fear'],df_loi['disinterest'])
coh_d_fea_loi1 = hp.cohen_d(df_emotion['fear'],df_loi['disinterest'])
corr_fea_loi2 = hp.correlation(df_emotion['fear'],df_loi['normal'])
coh_d_fea_loi2 = hp.cohen_d(df_emotion['fear'],df_loi['normal'])
corr_fea_loi3 = hp.correlation(df_emotion['fear'],df_loi['high interest'])
coh_d_fea_loi3 = hp.cohen_d(df_emotion['fear'],df_loi['high interest'])

##########
# happiness
print('HAPPINESS')
print('Affect and happiness')
corr_hap_a_aggressiv = hp.correlation(df_emotion['happiness'],df_affect['aggressiv'])
coh_d_hap_a_aggressiv = hp.cohen_d(df_emotion['happiness'],df_affect['aggressiv'])
corr_hap_a_cheerful = hp.correlation(df_emotion['happiness'],df_affect['cheerful'])
coh_d_hap_a_cheerful = hp.cohen_d(df_emotion['happiness'],df_affect['cheerful'])
corr_hap_a_intoxicated = hp.correlation(df_emotion['happiness'],df_affect['intoxicated'])
coh_d_hap_a_intoxicated = hp.cohen_d(df_emotion['happiness'],df_affect['intoxicated'])
corr_hap_a_nervous = hp.correlation(df_emotion['happiness'],df_affect['nervous'])
coh_d_hap_a_nervous = hp.cohen_d(df_emotion['happiness'],df_affect['nervous'])
corr_hap_a_neutral = hp.correlation(df_emotion['happiness'],df_affect['neutral'])
coh_d_hap_a_neutral = hp.cohen_d(df_emotion['happiness'],df_affect['neutral'])
corr_hap_a_tired = hp.correlation(df_emotion['happiness'],df_affect['tired'])
coh_d_hap_a_tired = hp.cohen_d(df_emotion['happiness'],df_affect['tired'])

# Now level of Interest
print('Level of Interest and happiness')
corr_hap_loi1 = hp.correlation(df_emotion['happiness'],df_loi['disinterest'])
coh_d_hap_loi1 = hp.cohen_d(df_emotion['happiness'],df_loi['disinterest'])
corr_hap_loi2 = hp.correlation(df_emotion['happiness'],df_loi['normal'])
coh_d_hap_loi2 = hp.cohen_d(df_emotion['happiness'],df_loi['normal'])
corr_hap_loi3 = hp.correlation(df_emotion['happiness'],df_loi['high interest'])
coh_d_hap_loi3 = hp.cohen_d(df_emotion['happiness'],df_loi['high interest'])

##########
# neutral
print('NEUTRAL')
print('Affect and neutral')
corr_emoneu_a_aggressiv = hp.correlation(df_emotion['neutral'],df_affect['aggressiv'])
coh_d_emoneu_a_aggressiv = hp.cohen_d(df_emotion['neutral'],df_affect['aggressiv'])
corr_emoneu_a_cheerful = hp.correlation(df_emotion['neutral'],df_affect['cheerful'])
coh_d_emoneu_a_cheerful = hp.cohen_d(df_emotion['neutral'],df_affect['cheerful'])
corr_emoneu_a_intoxicated = hp.correlation(df_emotion['neutral'],df_affect['intoxicated'])
coh_d_emoneu_a_intoxicated = hp.cohen_d(df_emotion['neutral'],df_affect['intoxicated'])
corr_emoneu_a_nervous = hp.correlation(df_emotion['neutral'],df_affect['nervous'])
coh_d_emoneu_a_nervous = hp.cohen_d(df_emotion['neutral'],df_affect['nervous'])
corr_emoneu_a_neutral = hp.correlation(df_emotion['neutral'],df_affect['neutral'])
coh_d_emoneu_a_neutral = hp.cohen_d(df_emotion['neutral'],df_affect['neutral'])
corr_emoneu_tired = hp.correlation(df_emotion['neutral'],df_affect['tired'])
coh_d_emoneu_a_tired = hp.cohen_d(df_emotion['neutral'],df_affect['tired'])

# Now arousal and level of Interest
print('Level of Interest and neutral')
corr_emoneu_loi1 = hp.correlation(df_emotion['neutral'], df_loi['disinterest'])
coh_d_emoneu_loi1 = hp.cohen_d(df_emotion['neutral'], df_loi['disinterest'])
corr_emoneu_loi2 = hp.correlation(df_emotion['neutral'], df_loi['normal'])
coh_d_emoneu_loi2 = hp.cohen_d(df_emotion['neutral'], df_loi['normal'])
corr_emoneu_loi3 = hp.correlation(df_emotion['neutral'], df_loi['high interest'])
coh_d_emoneu_loi3 = hp.cohen_d(df_emotion['neutral'], df_loi['high interest'])


##########
# Sadness
print('SADNESS')
print('Affect and sadness')
corr_sad_a_aggressiv = hp.correlation(df_emotion['sadness'],df_affect['aggressiv'])
coh_d_sad_a_aggressiv = hp.cohen_d(df_emotion['sadness'],df_affect['aggressiv'])
corr_sad_a_cheerful = hp.correlation(df_emotion['sadness'],df_affect['cheerful'])
coh_d_sad_a_cheerful = hp.cohen_d(df_emotion['sadness'],df_affect['cheerful'])
corr_sad_a_intoxicated = hp.correlation(df_emotion['sadness'],df_affect['intoxicated'])
coh_d_sad_a_intoxicated = hp.cohen_d(df_emotion['sadness'],df_affect['intoxicated'])
corr_sad_a_nervous = hp.correlation(df_emotion['sadness'],df_affect['nervous'])
coh_d_sad_a_nervous = hp.cohen_d(df_emotion['sadness'],df_affect['nervous'])
corr_sad_a_neutral = hp.correlation(df_emotion['sadness'],df_affect['neutral'])
coh_d_sad_a_neutral = hp.cohen_d(df_emotion['sadness'],df_affect['neutral'])
corr_sad_tired = hp.correlation(df_emotion['sadness'],df_affect['tired'])
coh_d_sad_a_tired = hp.cohen_d(df_emotion['sadness'],df_affect['tired'])

# Now level of Interest
print('Level of Interest and sadness')
corr_sad_loi1 = hp.correlation(df_emotion['sadness'],df_loi['disinterest'])
coh_d_sad_loi1 = hp.cohen_d(df_emotion['sadness'],df_loi['disinterest'])
corr_sad_loi2 = hp.correlation(df_emotion['sadness'],df_loi['normal'])
coh_d_sad_loi2 = hp.cohen_d(df_emotion['sadness'],df_loi['normal'])
corr_sad_loi3 = hp.correlation(df_emotion['sadness'],df_loi['high interest'])
coh_d_sad_loi3 = hp.cohen_d(df_emotion['sadness'],df_loi['high interest'])

ANGER
Affect and anger
Correlation between anger and aggressiv: 0.8219437272699538
Cohen d: -2.5220828261171917
Correlation between anger and cheerful: 0.8336310706800888
Cohen d: -3.756066144914373
Correlation between anger and intoxicated: -0.7093611149907214
Cohen d: -2.0544981822259185
Correlation between anger and nervous: 0.4679141958102869
Cohen d: -3.899662916493322
Correlation between anger and neutral: -0.275407946565424
Cohen d: -1.6917353317472992
Correlation between anger and tired: 0.30124420924847395
Cohen d: -1.203980867797832
Level of Interest and anger
Correlation between anger and disinterest: 0.009259088185940937
Cohen d: -0.7916027783022161
Correlation between anger and normal: 0.42912660568043326
Cohen d: -2.3772808250222512
Correlation between anger and high interest: -0.4572046402007513
Cohen d: -1.4269188233126726
BOREDOM
Affect and boredom
Correlation between boredom and aggressiv: -0.13264841610321568
Cohen d: -0.24405618151390246
Correlation between boredom 

### Now have a look at affect

In [7]:
##########
# Aggressiv
print('AGGRESSIV')
print('Level of Interest and aggressiv')
corr_agg_loi1 = hp.correlation(df_affect['aggressiv'],df_loi['disinterest'])
coh_d_agg_loi1 = hp.cohen_d(df_affect['aggressiv'],df_loi['disinterest'])
corr_agg_loi2 = hp.correlation(df_affect['aggressiv'],df_loi['normal'])
coh_d_agg_loi2 = hp.cohen_d(df_affect['aggressiv'],df_loi['normal'])
corr_agg_loi3 = hp.correlation(df_affect['aggressiv'],df_loi['high interest'])
coh_d_agg_loi3 = hp.cohen_d(df_affect['aggressiv'],df_loi['high interest'])


##########
# Cheerful
print('CHEERFUL')
print('Level of Interest and cheerful')
corr_che_loi1 = hp.correlation(df_affect['cheerful'],df_loi['disinterest'])
coh_d_che_loi1 = hp.cohen_d(df_affect['cheerful'],df_loi['disinterest'])
corr_che_loi2 = hp.correlation(df_affect['cheerful'],df_loi['normal'])
coh_d_che_loi2 = hp.cohen_d(df_affect['cheerful'],df_loi['normal'])
corr_che_loi3 = hp.correlation(df_affect['cheerful'],df_loi['high interest'])
coh_d_che_loi3 = hp.cohen_d(df_affect['cheerful'],df_loi['high interest'])

##########
# Intoxicated
print('INTOXICATED')
print('Level of Interest and intoxicated')
corr_tox_loi1 = hp.correlation(df_affect['intoxicated'],df_loi['disinterest'])
coh_d_tox_loi1 = hp.cohen_d(df_affect['intoxicated'],df_loi['disinterest'])
corr_tox_loi2 = hp.correlation(df_affect['intoxicated'],df_loi['normal'])
coh_d_tox_loi2 = hp.cohen_d(df_affect['intoxicated'],df_loi['normal'])
corr_tox_loi3 = hp.correlation(df_affect['intoxicated'],df_loi['high interest'])
coh_d_tox_loi3 = hp.cohen_d(df_affect['intoxicated'],df_loi['high interest'])

##########
# Nervous
print('NERVOUS')
print('Level of Interest and nervous')
corr_ner_loi1 = hp.correlation(df_affect['nervous'],df_loi['disinterest'])
coh_d_new_loi1 = hp.cohen_d(df_affect['nervous'],df_loi['disinterest'])
corr_ner_loi2 = hp.correlation(df_affect['nervous'],df_loi['normal'])
coh_d_new_loi2 = hp.cohen_d(df_affect['nervous'],df_loi['normal'])
corr_ner_loi3 = hp.correlation(df_affect['nervous'],df_loi['high interest'])
coh_d_new_loi3 = hp.cohen_d(df_affect['nervous'],df_loi['high interest'])

##########
# Neutral
print('NEUTRAL')
print('Level of Interest and neutral')
# Now intoxicated and level of Interest
corr_affneu_loi1 = hp.correlation(df_affect['neutral'],df_loi['disinterest'])
coh_d_affneu_loi1 = hp.cohen_d(df_affect['neutral'],df_loi['disinterest'])
corr_affneu_loi2 = hp.correlation(df_affect['neutral'],df_loi['normal'])
coh_d_affneu_loi2 = hp.cohen_d(df_affect['neutral'],df_loi['normal'])
corr_affneu_loi3 = hp.correlation(df_affect['neutral'],df_loi['high interest'])
coh_d_affneu_loi3 = hp.cohen_d(df_affect['neutral'],df_loi['high interest'])

#########
# Tired
print('TIRED')
print('Level of Interest and tired')
corr_tir_loi1 = hp.correlation(df_affect['tired'],df_loi['disinterest'])
coh_d_tir_loi1 = hp.cohen_d(df_affect['tired'],df_loi['disinterest'])
corr_tir_loi2 = hp.correlation(df_affect['tired'],df_loi['normal'])
coh_d_tir_loi2 = hp.cohen_d(df_affect['tired'],df_loi['normal'])
corr_tir_loi3 = hp.correlation(df_affect['tired'],df_loi['high interest'])
coh_d_tir_loi3 = hp.cohen_d(df_affect['tired'],df_loi['high interest'])

AGGRESSIV
Level of Interest and aggressiv
Correlation between aggressiv and disinterest: -0.46985007624380576
Cohen d: 1.6685232998938873
Correlation between aggressiv and normal: 0.6493212625168995
Cohen d: -1.9559760973290863
Correlation between aggressiv and high interest: -0.6382560988794522
Cohen d: -0.9907769562863176
CHEERFUL
Level of Interest and cheerful
Correlation between cheerful and disinterest: 0.30740629850886686
Cohen d: 3.2578193931742008
Correlation between cheerful and normal: 0.18631170633269223
Cohen d: -1.255894587971958
Correlation between cheerful and high interest: -0.23207811608422513
Cohen d: -0.2755757861703259
INTOXICATED
Level of Interest and intoxicated
Correlation between intoxicated and disinterest: -0.5066896737629814
Cohen d: 1.8942451153533637
Correlation between intoxicated and normal: -0.18317122995961105
Cohen d: -0.9401051861079537
Correlation between intoxicated and high interest: 0.25079466125469346
Cohen d: -0.06266537498593241
NERVOUS
Level o

## Now look at character data

## Chi-squared Test of Independence
We Start with characteristic sex. The null hypothesis states that the two categorical variables sex and e.g. emotion are independent.

For that we convert the previously used data frames which contain probabilities of e.g. emotion, affect etc. to frequency tables using the helper method calcFrequencyTable(). For each row (sample) the function selects the maximum probability and counts it as an occured frequency. Furthermore the function takes in an integer so that the matching labels for emotion are generated (voice features). The second integer defines the character feature e.g. Age, so that it can split the data set into a form of n x m, where n stands for the number of character features (in most cases it's 2, but for age it's 3) and m stands for the number of voice features e.g. number of different emotions.  
The frequency tables are then used as input for the stats.chi2_contingency() function.

If the p-value is significant, residuals are computed to identify the cells, in which the groups differ, i.e. the data sets differ in the emotion 'anger', which contributes to the significant p-value.

In [8]:
#Emotion
df_emotion_char_male_clean = hp.cleanData(df_emotion_char, 'Sex', 0)
df_emotion_char_female_clean = hp.cleanData(df_emotion_char, 'Sex', 1)
emotion_sex_fre = hp.calcFrequencyTable([df_emotion_char_male_clean, df_emotion_char_female_clean], 0, 0)
emotion_sex_fre.index = ['Male', 'Female']
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
emotion_sex_fre += 5
chi2_sex_emotion = st.chi2_contingency(emotion_sex_fre)
print('Chi square of emotion: ' + str(chi2_sex_emotion[0]) + ' with p-value of: ' + str(chi2_sex_emotion[1]))
emotion_sex_table = sm.stats.Table(emotion_sex_fre)
emotion_sex_res = emotion_sex_table.standardized_resids

#Affect
df_affect_char_male_clean = hp.cleanData(df_affect_char, 'Sex', 0)
df_affect_char_female_clean = hp.cleanData(df_affect_char, 'Sex', 1)
affect_sex_fre = hp.calcFrequencyTable([df_affect_char_male_clean, df_affect_char_female_clean], 1, 0)
affect_sex_fre.index = ['Male', 'Female']
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
affect_sex_fre += 5
chi2_sex_affect = st.chi2_contingency(affect_sex_fre)
print('Chi square of affect: ' + str(chi2_sex_affect[0])+  ' with p-value of: ' + str(chi2_sex_affect[1]))
affect_sex_table = sm.stats.Table(affect_sex_fre)
affect_sex_res = affect_sex_table.standardized_resids

#Arousal/Valence
df_ar_val_char_male_clean = hp.cleanData(df_ar_val_char, 'Sex', 0)
df_ar_val_char_female_clean = hp.cleanData(df_ar_val_char, 'Sex', 1)
ar_val_sex_fre = hp.calcFrequencyTable([df_ar_val_char_male_clean, df_ar_val_char_female_clean], 2, 0)
ar_val_sex_fre.index = ['Male', 'Female']
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
ar_val_sex_fre += 5
chi2_sex_ar_val = st.chi2_contingency(ar_val_sex_fre)
print('Chi square of arousal valence: ' + str(chi2_sex_ar_val[0]) + ' with p-value of: ' + str(chi2_sex_ar_val[1]))
ar_val_sex_table = sm.stats.Table(ar_val_sex_fre)
ar_val_sex_res = ar_val_sex_table.standardized_resids

#Level of Interest
df_loi_char_male_clean = hp.cleanData(df_loi_char, 'Sex', 0)
df_loi_char_female_clean = hp.cleanData(df_loi_char, 'Sex', 1)
loi_sex_fre = hp.calcFrequencyTable([df_loi_char_male_clean,df_loi_char_female_clean],3,0)
loi_sex_fre.index = ['Male', 'Female']
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
loi_sex_fre += 5
chi2_sex_loi = st.chi2_contingency(ar_val_sex_fre)
print('Chi square of Level of Interest: ' + str(chi2_sex_loi[0]) + ' with p-value of: ' + str(chi2_sex_loi[1]))
loi_sex_table = sm.stats.Table(loi_sex_fre)
loi_sex_res = loi_sex_table.standardized_resids

Chi square of emotion: 0.23324423324423343 with p-value of: 0.9997577204712553
Chi square of affect: 0.10732466063348399 with p-value of: 0.999806812314343
Chi square of arousal valence: 0.11349206349206363 with p-value of: 0.7362029664490812
Chi square of Level of Interest: 0.11349206349206363 with p-value of: 0.7362029664490812


Now move on to academic status, the hypothesis being that the variables academic status and e.g. emotion are independent.

In [9]:
#Emotion
df_emotion_char_bach_clean = hp.cleanData(df_emotion_char, 'Academic Status', 0)
df_emotion_char_mast_clean = hp.cleanData(df_emotion_char, 'Academic Status', 1)
emotion_aca_fre = hp.calcFrequencyTable([df_emotion_char_bach_clean, df_emotion_char_mast_clean], 0, 1)
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
emotion_aca_fre += 5
chi2_aca_emotion = st.chi2_contingency(emotion_aca_fre)
print('Chi square of emotion: ' + str(chi2_aca_emotion[0]) + ' with p-value of: ' + str(chi2_aca_emotion[1]))
emotion_aca_table = sm.stats.Table(emotion_aca_fre)
emotion_aca_res = emotion_aca_table.standardized_resids

#Affect
df_affect_char_bach_clean = hp.cleanData(df_affect_char, 'Academic Status', 0)
df_affect_char_mast_clean = hp.cleanData(df_affect_char, 'Academic Status', 1)
affect_aca_fre = hp.calcFrequencyTable([df_affect_char_bach_clean, df_affect_char_mast_clean], 1, 1)
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
affect_aca_fre += 5
chi2_aca_affect = st.chi2_contingency(affect_aca_fre)
print('Chi square of affect: ' + str(chi2_aca_affect[0])+  ' with p-value of: ' + str(chi2_aca_affect[1]))
affect_aca_table = sm.stats.Table(affect_aca_fre)
affect_aca_res = affect_aca_table.standardized_resids

#Arousal/Valence
df_ar_val_char_bach_clean = hp.cleanData(df_ar_val_char, 'Academic Status', 0)
df_ar_val_char_mast_clean = hp.cleanData(df_ar_val_char, 'Academic Status', 1)
ar_val_aca_fre = hp.calcFrequencyTable([df_ar_val_char_bach_clean, df_ar_val_char_mast_clean], 2, 1)
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
ar_val_aca_fre += 5
chi2_aca_ar_val = st.chi2_contingency(ar_val_aca_fre)
print('Chi square of arousal valence: ' + str(chi2_aca_ar_val[0]) + ' with p-value of: ' + str(chi2_aca_ar_val[1]))
ar_val_aca_table = sm.stats.Table(ar_val_aca_fre)
ar_val_aca_res = ar_val_aca_table.standardized_resids

#Level of Interest
df_loi_char_bach_clean = hp.cleanData(df_loi_char, 'Academic Status', 0)
df_loi_char_mast_clean = hp.cleanData(df_loi_char, 'Academic Status', 1)
loi_aca_fre = hp.calcFrequencyTable([df_loi_char_bach_clean,df_loi_char_mast_clean],3,1)
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
loi_aca_fre += 5
chi2_aca_loi = st.chi2_contingency(loi_aca_fre)
print('Chi square of Level of Interest: ' + str(chi2_aca_loi[0]) + ' with p-value of: ' + str(chi2_aca_loi[1]))
loi_aca_table = sm.stats.Table(loi_aca_fre)
loi_aca_res = loi_aca_table.standardized_resids

Chi square of emotion: 0.23324423324423338 with p-value of: 0.9997577204712553
Chi square of affect: 0.7232748868778277 with p-value of: 0.9816669156305917
Chi square of arousal valence: 0.11349206349206363 with p-value of: 0.7362029664490812
Chi square of Level of Interest: 0.22291021671826644 with p-value of: 0.8945315474908987


Now let's look if age and e.g. emotion/ affect/ arousal-valence/ level of interest are independent

In [10]:
#Emotion
df_emotion_char_young_clean = hp.cleanData(df_emotion_char, 'Age', 23)
df_emotion_char_middle_clean = hp.cleanData(df_emotion_char, 'Age', 24)
df_emotion_char_old_clean = hp.cleanData(df_emotion_char, 'Age', 25)
emotion_age_fre = hp.calcFrequencyTable([df_emotion_char_young_clean, df_emotion_char_middle_clean, df_emotion_char_old_clean], 0, 2)
emotion_age_fre.index = ['Young', 'Middle', 'Old']
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
emotion_age_fre += 5
chi2_age_emotion = st.chi2_contingency(emotion_age_fre)
print('Chi square of emotion: ' + str(chi2_age_emotion[0]) + ' with p-value of: ' + str(chi2_age_emotion[1]))


#Affect
df_affect_char_young_clean = hp.cleanData(df_affect_char, 'Age', 23)
df_affect_char_middle_clean = hp.cleanData(df_affect_char, 'Age', 24)
df_affect_char_old_clean = hp.cleanData(df_affect_char, 'Age', 25)
affect_age_fre = hp.calcFrequencyTable([df_affect_char_young_clean, df_affect_char_middle_clean, df_affect_char_old_clean], 1, 2)
affect_age_fre.index = ['Young', 'Middle', 'Old']
############
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
affect_age_fre += 5
chi2_age_affect = st.chi2_contingency(affect_age_fre)
print('Chi square of affect: ' + str(chi2_age_affect[0])+  ' with p-value of: ' + str(chi2_age_affect[1]))

#Arousal/Valence
df_ar_val_char_young_clean = hp.cleanData(df_ar_val_char, 'Age', 23)
df_ar_val_char_middle_clean = hp.cleanData(df_ar_val_char, 'Age', 24)
df_ar_val_char_old_clean = hp.cleanData(df_ar_val_char, 'Age', 25)
ar_val_age_fre = hp.calcFrequencyTable([df_ar_val_char_young_clean, df_ar_val_char_middle_clean,df_ar_val_char_old_clean], 2, 2)
ar_val_age_fre.index = ['Young', 'Middle', 'Old']
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
ar_val_age_fre += 5
chi2_age_ar_val = st.chi2_contingency(ar_val_age_fre)
print('Chi square of arousal valence: ' + str(chi2_age_ar_val[0]) + ' with p-value of: ' + str(chi2_age_ar_val[1]))

#Level of Interest
df_loi_char_young_clean = hp.cleanData(df_loi_char, 'Age', 23)
df_loi_char_middle_clean = hp.cleanData(df_loi_char, 'Age', 24)
df_loi_char_old_clean = hp.cleanData(df_loi_char, 'Age', 25)
loi_age_fre = hp.calcFrequencyTable([df_loi_char_young_clean,df_loi_char_middle_clean, df_loi_char_old_clean],3,2)
loi_age_fre.index = ['Young', 'Middle', 'Old']
###########
#The following line is only there to account for zeroes in table, which lead to chisquare failing yielding warnings and errors
##########
loi_age_fre += 5
chi2_age_loi = st.chi2_contingency(loi_age_fre)
print('Chi square of Level of Interest: ' + str(chi2_age_loi[0]) + ' with p-value of: ' + str(chi2_age_loi[1]))

Chi square of emotion: 0.22291021671826622 with p-value of: 0.9999999975799772
Chi square of affect: 0.5759803921568627 with p-value of: 0.9999870029958444
Chi square of arousal valence: 0.22499999999999998 with p-value of: 0.8935973471085157
Chi square of Level of Interest: 0.22291021671826625 with p-value of: 0.9942316580471601


## Post-Hoc tests for age, as it has three different groups

In [13]:
print('post-hoc emotions and different groups')
emo_reject_list, emo_corrected_p_vals, emo_combinations, emo_residuals= hp.chi2_post_hoc(emotion_age_fre, 'bonferroni', True, True)
print('\n post-hoc affect and different groups')
aff_reject_list, emo_corrected_p_vals, emo_combinations, aff_residuals = hp.chi2_post_hoc(affect_age_fre, 'bonferroni', True, True)
print('\n post-hoc arousal-valence and different groups')
ar_val_reject_list, ar_val_corrected_p_vals, ar_val_combinations, ar_val_residuals = hp.chi2_post_hoc(ar_val_age_fre, 'bonferroni',True, True)
print('\n post-hoc level of intereset and different groups')
loi_reject_list, loi_corrected_p_vals, loi_combinations, loi_residuals = hp.chi2_post_hoc(loi_age_fre, 'bonferroni', True, True)

post-hoc emotions and different groups
Combinations: [('Young', 'Middle'), ('Young', 'Old'), ('Middle', 'Old')]
Reject List: [False False False]
Corrected p-values: [1. 1. 1.]

 post-hoc affect and different groups
Combinations: [('Young', 'Middle'), ('Young', 'Old'), ('Middle', 'Old')]
Reject List: [False False False]
Corrected p-values: [1. 1. 1.]

 post-hoc arousal-valence and different groups
Combinations: [('Young', 'Middle'), ('Young', 'Old'), ('Middle', 'Old')]
Reject List: [False False False]
Corrected p-values: [1. 1. 1.]

 post-hoc level of intereset and different groups
Combinations: [('Young', 'Middle'), ('Young', 'Old'), ('Middle', 'Old')]
Reject List: [False False False]
Corrected p-values: [1. 1. 1.]
